In [3]:
# Gene Expression Cancer RNA-Seq - Initial Data Inspection
# --------------------------------------------------------
# Goal: Explore high-dimensional RNA-Seq dataset for cancer classification.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

os.makedirs("figures", exist_ok=True)

np.random.seed(42)

# ============================
# 1. Data Acquisition & Loading
# ============================
data_path = "/Users/jeremygoetschy/Projects/RNA-SEQ/Raw/data.csv"
labels_path = "/Users/jeremygoetschy/Projects/RNA-SEQ/Raw/labels.csv"

data = pd.read_csv(data_path, index_col=0)  # gene expression matrix
labels = pd.read_csv(labels_path, index_col=0)  # target labels

print(f"Data shape: {data.shape}")  # (801, 20531)
print(f"Labels shape: {labels.shape}")  # (801, 1)

# Merge data & labels
df = data.join(labels, how="inner")

# ============================
# 2. Initial Inspection
# ============================
print(f"\nCombined shape: {df.shape}")  # (801, 20532)
print("\nDataFrame Head:")
print(df.head())

print("\nDataFrame Info:")
print(df.info())

print("\nDescriptive Statistics:")
print(df.describe())

print("\nData Types:")
print(df.dtypes.value_counts())

print("\nMissing Values:")
print(df.isnull().sum().sum())  # Expect 0

print("\nDuplicated Rows:")
print(df.duplicated().sum())  # Expect 0

print("\nNegative Values in Numeric Columns:")
print((df.select_dtypes(include=[np.number]) < 0).sum().sum())  # Expect 0

# ============================
# 3. Memory Optimization
# ============================
df = df.astype(
    {col: "float32" for col in df.select_dtypes(include=[np.float64]).columns}
)
print("\nUpdated Data Types (after downcasting):")
print(df.dtypes.value_counts())

# 🔎 Observations:
# - Dataset: 801 samples, 20,531 gene expression features + 1 label column.
# - Features are continuous (gene expression levels).
# - Target labels are categorical (cancer types).
# - No missing, duplicates, or negative values detected.
# - Converted float64 → float32 to optimize memory (critical with high-dimensional data).

Data shape: (801, 20531)
Labels shape: (801, 1)

Combined shape: (801, 20532)

DataFrame Head:
          gene_0    gene_1    gene_2    gene_3     gene_4  gene_5    gene_6  \
sample_0     0.0  2.017209  3.265527  5.478487  10.431999     0.0  7.175175   
sample_1     0.0  0.592732  1.588421  7.586157   9.623011     0.0  6.816049   
sample_2     0.0  3.511759  4.327199  6.881787   9.870730     0.0  6.972130   
sample_3     0.0  3.663618  4.507649  6.659068  10.196184     0.0  7.843375   
sample_4     0.0  2.655741  2.821547  6.539454   9.738265     0.0  6.566967   

            gene_7  gene_8  gene_9  ...  gene_20522  gene_20523  gene_20524  \
sample_0  0.591871     0.0     0.0  ...    8.210257    9.723516    7.220030   
sample_1  0.000000     0.0     0.0  ...    7.323865    9.740931    6.256586   
sample_2  0.452595     0.0     0.0  ...    8.127123   10.908640    5.401607   
sample_3  0.434882     0.0     0.0  ...    8.792959   10.141520    8.942805   
sample_4  0.360982     0.0     0.0 

In [4]:
# Gene Expression Cancer RNA-Seq - Exploratory Data Analysis (EDA)
# ----------------------------------------------------------------
# Goal: Explore target distribution, gene expression distributions,
#       skewness, correlations, and dimensionality insights.

import plotly.express as px
from scipy.stats import entropy, skew
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")


# ============================
# 1. Class Distribution
# ============================
class_counts = df["Class"].value_counts()
print("\nClass counts:")
print(class_counts)

# Imbalance entropy (normalized)
class_probs = class_counts / class_counts.sum()
imbalance_entropy = entropy(class_probs) / np.log(len(class_counts))
print(f"\nClass imbalance entropy: {imbalance_entropy:.3f}")
# → 0.939 → Mild imbalance.

fig = px.bar(
    x=class_counts.index,
    y=class_counts.values,
    labels={"x": "Class", "y": "Count"},
    title="Class Distribution",
)
fig.update_layout(showlegend=False)
fig.show()

# 🔎 Observations:
# - Dataset is relatively balanced across classes (entropy ~0.94).
# - Stratified K-Folds recommended for cross-validation.
# - SMOTE could still be applied if imbalance impacts modeling.


# ============================
# 2. Gene Expression Distributions
# ============================
sample_genes = df.drop("Class", axis=1).columns[:10]  # sample 10 genes

plt.figure(figsize=(12, 6))
for i, gene in enumerate(sample_genes):
    plt.subplot(2, 5, i + 1)
    sns.histplot(x=df[gene], bins=30, kde=True)
    plt.title(f"Gene: {gene}")
plt.tight_layout()
plt.savefig("figures/gene_expression_distributions.png")
plt.close()

# 🔎 Observations:
# - Gene distributions vary widely across samples.
# - Several genes contain many zeros → potential for log1p transform before scaling.


# ============================
# 3. Skewness of Gene Expressions
# ============================
skewness = df.drop("Class", axis=1).apply(skew)
print("\nSkewness of gene expression levels:")
print(f"Mean: {skewness.mean():.2f}, Median: {skewness.median():.2f}")

plt.figure(figsize=(12, 6))
sns.histplot(data=skewness, bins=30, kde=True)
plt.title("Skewness Distribution of Gene Expression Levels")
plt.xlabel("Skewness")
plt.ylabel("Density")
plt.savefig("figures/skewness_distribution.png")
plt.close()

# 🔎 Observations:
# - Average skewness = 0.92 (positively skewed).
# - Median skewness = 0.15 → majority of genes are close to symmetric, but
#   some have heavy right tails.
# - Suggests log-transformation could help normalization.


# ============================
# 4. Correlation Analysis
# ============================
sample_genes = df.drop("Class", axis=1).columns[:100]  # sample 100 genes
corr_matrix = df[sample_genes].corr(method="spearman")  # robust to non-linearities

plt.figure(figsize=(12, 10))
sns.heatmap(
    corr_matrix,
    cmap="coolwarm",
    annot=False,
    fmt=".2f",
    vmin=-1,
    vmax=1,
    square=True,
    cbar_kws={"shrink": 0.8},
)
plt.title("Correlation Heatmap of 100 Sampled Genes")
plt.savefig("figures/correlation_heatmap.png")
plt.close()

high_corr = (corr_matrix.abs() > 0.8).sum().sum() - len(corr_matrix)
print(f"Number of high correlations > 0.8: {high_corr//2}")

# 🔎 Observations:
# - Only a small number of gene pairs show high correlation (>0.8).
# - However, given the high dimensionality (~20k features), multicollinearity is expected.
# - Dimensionality reduction (PCA, t-SNE, autoencoders) may help extract signal.


Class counts:
Class
BRCA    300
KIRC    146
LUAD    141
PRAD    136
COAD     78
Name: count, dtype: int64

Class imbalance entropy: 0.939



Skewness of gene expression levels:
Mean: 0.92, Median: 0.15
Number of high correlations > 0.8: 3


In [5]:
# Gene Expression Cancer RNA-Seq - Dimensionality Reduction & Feature Importance
# -------------------------------------------------------------------------------
# Goal: Visualize high-dimensional RNA-Seq data (PCA, t-SNE) and extract top genes
#       with Random Forest feature importance.

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier

# ============================
# 1. Scaling
# ============================
X_viz = df.drop("Class", axis=1)
scaler = StandardScaler()
X_viz_scaled = scaler.fit_transform(X_viz)
y = LabelEncoder().fit_transform(df["Class"])


# ============================
# 2. PCA (Global Structure)
# ============================
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_viz_scaled)

print(f"PCA explained variance ratio: {pca.explained_variance_ratio_.sum():.3f}")

fig_pca = px.scatter(
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    color=df["Class"],
    title="PCA of Gene Expression Data",
    labels={"x": "PCA 1", "y": "PCA 2"},
)
fig_pca.show()

# 🔎 Observations:
# - PCA (2D) explains ~19% of total variance → limited dimensionality capture.
# - Some overlap between classes; only certain cancer types (e.g., KIRC) partially separate.
# - PCA is useful for variance structure but not sufficient for clustering.


# ============================
# 3. t-SNE (Local Structure)
# ============================
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X_viz_scaled)

fig_tsne = px.scatter(
    x=X_tsne[:, 0],
    y=X_tsne[:, 1],
    color=df["Class"],
    title="t-SNE of Gene Expression Data",
    labels={"x": "t-SNE 1", "y": "t-SNE 2"},
)
fig_tsne.show()

# 🔎 Observations:
# - t-SNE reveals **clear separation of cancer subtypes**.
# - Preserves local structure, showing transcriptomic profiles are distinct.
# - t-SNE is only for visualization, not feature input for ML models.


# ============================
# 4. Feature Importance (Random Forest)
# ============================
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_viz_scaled, y)

importances = rf.feature_importances_
top_genes = pd.Series(importances, index=X_viz.columns).sort_values(ascending=False)[
    :10
]

print("\nTop 10 Genes by Importance:")
print(top_genes)

fig = px.bar(
    top_genes,
    x=top_genes.index,
    y=top_genes.values,
    title="Top 10 Genes by Random Forest Importance",
    labels={"x": "Genes", "y": "Importance"},
)
fig.show()

# 🔎 Observations:
# - Top genes contribute ~0.7–1.3% each.
# - No single dominant gene → classification depends on a **multi-gene signature**.
# - Consistent with biological reality (cancer classification relies on multiple pathways).


# ============================
# 5. Outlier Detection (z-score)
# ============================
z_scores = np.abs(X_viz_scaled)
outliers = (z_scores > 3).sum(axis=1)

print(f"\nSamples with >10 extreme gene expressions: {(outliers > 10).sum()}")

# 🔎 Observations:
# - Many samples flagged as "outliers" due to high dimensionality (~20k genes).
# - Not true anomalies → reflects natural biological variability in RNA-Seq data.

PCA explained variance ratio: 0.193



Top 10 Genes by Importance:
gene_14092    0.013081
gene_15987    0.010757
gene_8099     0.009729
gene_7964     0.009284
gene_7896     0.009193
gene_5578     0.008598
gene_1122     0.008332
gene_16169    0.008284
gene_5598     0.007933
gene_14427    0.007054
dtype: float64



Samples with >10 extreme gene expressions: 801


In [6]:
# Gene Expression Cancer RNA-Seq - Preprocessing
# ----------------------------------------------
# Goal: Build a preprocessing pipeline to handle high-dimensional RNA-Seq data.
# Steps:
#   1. Train/test split with stratification.
#   2. Log transform (optional).
#   3. Standard scaling.
#   4. Feature selection (variance threshold).
#   5. PCA dimensionality reduction (optional).

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_selection import VarianceThreshold

# ============================
# 1. Train/Test Split
# ============================
X = df.drop("Class", axis=1)
y = df["Class"]  # already encoded in previous step with LabelEncoder if needed

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


# ============================
# 2. PCA Variance Explained
# ============================
X_train_tpm = scaler.fit_transform(np.log1p(X_train))
pca_full = PCA().fit(X_train_tpm)
cum_var = np.cumsum(pca_full.explained_variance_ratio_)
n_components = np.where(cum_var >= 0.8)[0][0] + 1

print(f"Number of PCA components for 80% variance: {n_components}")
# → 168 components


# ============================
# 3. Preprocessing Pipeline
# ============================
log_transformer = FunctionTransformer(np.log1p, validate=False)

preprocessor = Pipeline(
    steps=[
        # ("log", log_transformer),            # optional: log1p transform
        ("scaler", StandardScaler()),  # normalize gene expression
        ("var", VarianceThreshold(threshold=0.1)),  # remove low-variance genes
        # ("pca", PCA(n_components=n_components, whiten=True)),  # optional PCA
    ]
)

# 🔎 Observations:
# - Log transform tested, but did not improve model performance → commented.
# - Variance threshold (0.1) filters out uninformative genes → reduces dimensionality.
# - PCA to 168 components explains 80% variance, but initial tests showed
#   tree-based models performed better without PCA (retain full feature space).

Train shape: (640, 20531), Test shape: (161, 20531)
Number of PCA components for 80% variance: 168


In [7]:
# Gene Expression Cancer RNA-Seq - Modeling & Evaluation
# ------------------------------------------------------
# Goal: Compare classifiers on high-dimensional RNA-Seq data using
#       cross-validation and interpret predictions with SHAP.

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import (
    f1_score,
    recall_score,
    balanced_accuracy_score,
    classification_report,
)
import shap

# ============================
# 1. Candidate Models
# ============================
models = {
    "Logistic Regression": LogisticRegression(penalty="l2", C=0.01, max_iter=1000),
    # "Random Forest": RandomForestClassifier(),
    # "XGBoost": XGBClassifier(),
    # "SVC": SVC(probability=True),
    # "LightGBM": LGBMClassifier(),
    # "CatBoost": CatBoostClassifier(verbose=0),
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    "f1_macro": "f1_macro",
    "f1_weighted": "f1_weighted",
    "recall_macro": "recall_macro",
    "balanced_acc": "balanced_accuracy",
}

# ============================
# 2. Cross-Validation Results
# ============================
results = {}
for name, model in models.items():
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])
    cv_results = cross_validate(
        pipeline, X_train, y_train, cv=cv, scoring=scoring, n_jobs=-1
    )
    results[name] = {
        "F1 Macro": cv_results["test_f1_macro"].mean(),
        "F1 Weighted": cv_results["test_f1_weighted"].mean(),
        "Recall Macro": cv_results["test_recall_macro"].mean(),
        "Balanced Accuracy": cv_results["test_balanced_acc"].mean(),
    }

results_df = pd.DataFrame(results).T
print("\nCross-Validation Results:")
print(results_df)

# Visualization
plt.figure(figsize=(18, 6))
for i, metric in enumerate(results_df.columns):
    ax = plt.subplot(1, 4, i + 1)
    sns.barplot(x=results_df.index, y=results_df[metric], ax=ax)
    ax.set_title(f"Model Comparison - {metric}")
    ax.set_ylabel(metric)
    ax.set_xlabel("Models")
    for j, v in enumerate(results_df[metric]):
        ax.text(j, v + 0.005, f"{v:.3f}", ha="center", va="bottom")
plt.tight_layout()
plt.savefig("figures/model_comparison.png")
plt.close()


# ============================
# 3. Final Model (Logistic Regression with PCA)
# ============================
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", LogisticRegression(penalty="l2", C=0.01, max_iter=1000)),
    ]
)
pipeline.fit(X_train, y_train)

# Transform data for SHAP
X_train_transformed = pipeline.named_steps["preprocessor"].transform(X_train)
X_test_transformed = pipeline.named_steps["preprocessor"].transform(X_test)

print(f"Train transformed shape: {X_train_transformed.shape}")
print(f"Test transformed shape: {X_test_transformed.shape}")


# ============================
# 4. Model Interpretation (SHAP)
# ============================
explainer = shap.LinearExplainer(pipeline.named_steps["model"], X_train_transformed)
shap_values = explainer(X_train_transformed)

shap.summary_plot(
    shap_values,
    X_train_transformed,
    feature_names=X.columns,
    plot_type="bar",
    show=False,
)
plt.savefig("figures/shap_summary.png")
plt.close()

# 🔎 Observations:
# - SHAP highlights the most predictive genes across PCA-reduced space.
# - Interpretation easier when using VarianceThreshold (original features).
# - Best accuracy achieved with PCA → tradeoff between interpretability and performance.


# ============================
# 5. Evaluation on Test Set
# ============================
y_pred = pipeline.predict(X_test)

print("\nTest Set Metrics")
print(f"F1 Macro: {f1_score(y_test, y_pred, average='macro'):.3f}")
print(f"F1 Weighted: {f1_score(y_test, y_pred, average='weighted'):.3f}")
print(f"Recall Macro: {recall_score(y_test, y_pred, average='macro'):.3f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred):.3f}")

print("\nClassification Report:")
print(
    classification_report(
        y_test, y_pred, target_names=pipeline.named_steps["model"].classes_
    )
)


# ============================
# 6. Cross-Validation Results (Reference with PCA=0.8)
# ============================
# Logistic Regression  → F1 Macro = 1.000
# Random Forest        → F1 Macro ≈ 0.982
# XGBoost              → F1 Macro ≈ 0.970
# SVC                  → F1 Macro ≈ 0.991
# LightGBM             → F1 Macro ≈ 0.980
# CatBoost             → F1 Macro ≈ 0.978
#
# 🔎 Notes:
# - Logistic Regression with PCA + L2 regularization achieves perfect separation (F1=1.0).
# - Tree-based models slightly below LR, but still excellent (>0.97).
# - PCA boosts generalization by reducing noise and multicollinearity.
# - Best performance: Logistic Regression (interpretable & high accuracy).

/opt/homebrew/Caskroom/miniconda/base/envs/ds_elite/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html




Cross-Validation Results:
                     F1 Macro  F1 Weighted  Recall Macro  Balanced Accuracy
Logistic Regression  0.998657     0.998427      0.998182           0.998182
Train transformed shape: (640, 20254)
Test transformed shape: (161, 20254)


/var/folders/db/5zyr74rj7fn21wgcyltjd3440000gn/T/ipykernel_9585/3144437243.py:100: FutureWarning:

The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.




Test Set Metrics
F1 Macro: 0.995
F1 Weighted: 0.994
Recall Macro: 0.993
Balanced Accuracy: 0.993

Classification Report:
              precision    recall  f1-score   support

        BRCA       0.98      1.00      0.99        60
        COAD       1.00      1.00      1.00        16
        KIRC       1.00      1.00      1.00        30
        LUAD       1.00      0.96      0.98        28
        PRAD       1.00      1.00      1.00        27

    accuracy                           0.99       161
   macro avg       1.00      0.99      0.99       161
weighted avg       0.99      0.99      0.99       161



In [8]:
# Gene Expression Cancer RNA-Seq - Top Gene Selection with SHAP
# -------------------------------------------------------------
# Goal: Select most predictive genes using VarianceThreshold + RandomForest + SHAP.

# ============================
# 1. Feature Selection (Variance Threshold)
# ============================
selector = VarianceThreshold(threshold=0.8)
X_train_var = selector.fit_transform(X_train)
X_test_var = selector.transform(X_test)

selected_genes = X.columns[selector.get_support()]
print(f"Conserved Genes after VarianceThreshold: {len(selected_genes)}")

# 🔎 Observations:
# - Filters out low-variance genes (uninformative across samples).
# - Reduced feature space significantly for downstream interpretation.


# ============================
# 2. Train Random Forest
# ============================
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf.fit(X_train_var, y_train)


# ============================
# 3. SHAP Importance
# ============================
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_train_var)

if isinstance(shap_values, list):  # multiclass output
    shap_abs_mean = np.mean([np.abs(sv).mean(axis=0) for sv in shap_values], axis=0)
else:
    shap_abs_mean = np.abs(shap_values).mean(axis=0)

print("Number of selected genes:", len(selected_genes))
print("Shape of SHAP values (aggregated):", shap_abs_mean.shape)

# Compute mean absolute SHAP value per gene
shap_abs_mean_per_gene = shap_abs_mean.mean(axis=1)

top_genes_df = pd.DataFrame(
    {"Gene": selected_genes, "SHAP Value": shap_abs_mean_per_gene}
).sort_values(by="SHAP Value", ascending=False)

# Save Top 100 genes
top_genes_df.head(100).to_csv("top_100_genes_shap.csv", index=False)
print("Top 100 genes saved to top_100_genes_shap.csv")


# ============================
# 4. SHAP Summary Plot
# ============================
shap.summary_plot(shap_values, feature_names=selected_genes, show=False)
plt.savefig("figures/shap_top_genes_summary.png")
plt.close()

# 🔎 Observations:
# - SHAP identifies the **most predictive genes** for cancer subtype classification.
# - Top genes contribute ~0.5–1.5% each → classification relies on multi-gene signatures.
# - Exported CSV provides a **gene signature of top 100 predictors**,
#   useful for downstream bioinformatics interpretation (e.g., pathway enrichment).

Conserved Genes after VarianceThreshold: 10234
Number of selected genes: 10234
Shape of SHAP values (aggregated): (10234, 5)
Top 100 genes saved to top_100_genes_shap.csv


/var/folders/db/5zyr74rj7fn21wgcyltjd3440000gn/T/ipykernel_9585/1439429044.py:56: FutureWarning:

The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.

/opt/homebrew/Caskroom/miniconda/base/envs/ds_elite/lib/python3.11/site-packages/shap/plots/_beeswarm.py:726: FutureWarning:

The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.

/opt/homebrew/Caskroom/miniconda/base/envs/ds_elite/lib/python3.11/site-packages/shap/plots/_beeswarm.py:746: FutureWarning:

The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.

/opt/homebrew/Caskro

<Figure size 640x480 with 0 Axes>